# Datos del equipo (COMPLETAR)

Identificador de grupo: Grupo XX.

| Nombre y apellido | % de contribución |
|-------------------|-------------------|
|                   |                   |
|                   |                   |
|                   |                   |
|                   |                   |
|                   |                   |


## 0. INSTALACIÓN Y CONFIGURACIÓN

In [ ]:
%pip install torch torchvision hypothesis adversarial-robustness-toolbox adversarial-robustness-toolbox[pytorch] scikit-learn matplotlib

## 1. CONFIGURACIÓN INICIAL Y CARGA DE LIBRERÍAS

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import datasets, transforms

import numpy as np
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import sklearn

from hypothesis import given, strategies as st, settings
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent

import warnings
warnings.filterwarnings('ignore')

# Reproducibilidad
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
sklearn.random.seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando: {device}")

## 2. DATOS Y MODELO BASELINE

In [ ]:
# type: ignore

# Cargar MNIST (versión simplificada, mitad del dataset)
transform = transforms.Compose([
    transforms.ToTensor()
])

train_full = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_full = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_indices = np.random.choice(len(train_full), 30000, replace=False)
test_indices = np.random.choice(len(test_full), 5000, replace=False)

train_subset = Subset(train_full, train_indices)
test_subset = Subset(test_full, test_indices)

train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=128, shuffle=False)

print(f"Train: {len(train_subset)}, Test: {len(test_subset)}")

In [ ]:
# Modelo simple
class SimpleNN(nn.Module):
    """Red neuronal simple para MNIST"""
    def __init__(self):
        super(SimpleNN, self).__init__()

        # Capa 1
        self.flatten = nn.Flatten()         # [1,28,28] → [784]
        self.fc1 = nn.Linear(28*28, 128)    # [784] → [128]
        self.relu1 = nn.ReLU()              # Activación no-lineal
        self.dropout1 = nn.Dropout(0.2)     # Regularización

        # Capa 2
        self.fc2 = nn.Linear(128, 64)   # [128] → [64]
        self.relu2 = nn.ReLU()          # Activación no-lineal
        self.dropout2 = nn.Dropout(0.2) # Regularización

        # Capa salida
        self.fc3 = nn.Linear(64, 10) # [64] → [10] (10 clases)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu2(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return x

model = SimpleNN().to(device)
print(f"Parámetros: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
def train_model(model, train_loader, epochs=3):
    """Entrena el modelo"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            if batch_idx % 100 == 0:
                print(f'  Batch {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}')

        acc = 100. * correct / total
        print(f'Época {epoch+1}: Loss: {running_loss/len(train_loader):.3f}, '
              f'Accuracy: {acc:.1f}%')

    return model

def evaluate_simple(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return 100 * correct / total

print("Entrenando modelo baseline...")
train_model(model, train_loader, epochs=5)
acc = evaluate_simple(model, test_loader)
print(f"\nAccuracy: {acc:.2f}%")

## 3. PROPERTY-BASED TESTING (I)

In [ ]:
# type: ignore

def predict_single(model, image):
    """Predice una imagen y retorna clase y confianza"""
    model.eval()
    with torch.no_grad():
        if len(image.shape) == 3:
            image = image.unsqueeze(0)
        image = image.to(device)
        output = model(image)
        probs = F.softmax(output, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0, pred_class].item()
    return pred_class, confidence

@given(st.integers(min_value=0, max_value=len(test_subset)-1))
@settings(max_examples=20, deadline=None)
def test_consistency(idx):
    # EJERCICIO 1: Verifica que dos predicciones de la misma imagen sean idénticas.
    # 1. Obtener imagen
    # 2. Predecir dos veces
    # 3. Verificar con assert
    # === INICIO SOLUCIÓN ===

    # === FIN SOLUCIÓN ===
    pass

# Ejecutar test
try:
    test_consistency()
    print("✅ Test de consistencia: PASADO")
except Exception as e:
    print(f"❌ Test fallado: {e}")

## 4. PROPERTY-BASED TESTING (II)

In [ ]:
# type: ignore

@given(
    st.integers(min_value=0, max_value=len(test_subset)-1),
    st.floats(min_value=0.01, max_value=0.3)
)
@settings(max_examples=15, deadline=None)
def test_noise_robustness(idx, noise_std):
    # EJERCICIO 2: Verifica que un pequeño ruido no destruya la confianza del modelo.
    # 1. Obtener imagen
    # 2. Crear imagen con ruido, por ejemplo: image_noisy = image + torch.randn_like(image) * noise_std
    # 3. Predecir ambas
    # 4. Verificar que la confianza no baja más de un 50% con respecto de la original
    # === INICIO SOLUCIÓN ===

    # === FIN SOLUCIÓN ===
    pass

try:
    test_noise_robustness()
    print("✅ Test de ruido: PASADO")
except Exception as e:
    print(f"⚠️ Test fallado: {str(e)[:80]}")

## 5. VALIDACIÓN ESTADÍSTICA

In [ ]:
# Obtener predicciones para bootstrap
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

def bootstrap_accuracy(preds, labels, n_iterations=1000):
    # EJERCICIO 3: Calcula intervalo de confianza 95% para accuracy.
    # 1. Crear lista vacía: accuracies = []
    # 2. Iterar n_iterations veces
    # 3. Resamplear: indices = resample(range(len(preds)), n_samples=len(preds))
    # 4. Calcular accuracy: acc = accuracy_score(labels[indices], preds[indices])
    # 5. Añadir accuracy a la lista
    # 6. Retornar tupla (media, percentil 2.5, percentil 97.5)
    # === INICIO SOLUCIÓN ===

    # === FIN SOLUCIÓN ===
    pass

# Ejecutar bootstrap
print("Calculando bootstrap...")
mean, lower, upper = bootstrap_accuracy(all_preds, all_labels, n_iterations=1000)
print(f"\nAccuracy: {mean*100:.2f}% [IC 95%: {lower*100:.2f}% - {upper*100:.2f}%]")
print(f"Ancho del intervalo: {(upper-lower)*100:.2f}%")

## 6. COMPARACIÓN CON BASELINE

In [ ]:
# Simulamos un segundo modelo (copia con ruido en predicciones)
# En la práctica real, serían predicciones de otro modelo
model2_preds = all_preds.copy()
# Simular que el modelo 2 comete algunos errores diferentes
flip_indices = np.random.choice(len(model2_preds), size=50, replace=False)
model2_preds[flip_indices] = (model2_preds[flip_indices] + 1) % 10

def compare_models(preds1, preds2, labels):
    # EJERCICIO 4: Compara estadísticamente dos modelos.
    # 1. Calcular bootstrap para modelo 1 con 500 iteraciones
    # 2. Calcular bootstrap para modelo 2 con 500 iteraciones
    # 3. Comprobar si intervalos NO se solapan (no_overlap)
    # 4. Retornar no_overlap y ambas medias
    # === INICIO SOLUCIÓN ===

    # === FIN SOLUCIÓN ===
    pass

print("Comparando modelos...")
different, acc1, acc2 = compare_models(all_preds, model2_preds, all_labels)
print(f"\nModelo 1: {acc1*100:.2f}%")
print(f"Modelo 2: {acc2*100:.2f}%")
print(f"¿Son significativamente diferentes? {'SÍ' if different else 'NO'}")

## 7. EVALUACIÓN ADVERSARIA

In [ ]:
# type: ignore

# Preparar datos sin normalización para ART
transform_art = transforms.ToTensor()
test_art = datasets.MNIST('./data', train=False, transform=transform_art)
test_subset_art = Subset(test_art, test_indices[:500])  # Solo 500 para rapidez

X_test = []
y_test = []
for img, label in test_subset_art:
    X_test.append(img.numpy())
    y_test.append(label)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(f"Datos para ART: {X_test.shape}")

In [ ]:
# Wrapper para ART (normaliza internamente)
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        # Normalizar
        x = (x - 0.1307) / 0.3081
        return self.model(x)

wrapped = ModelWrapper(model)

# Crear clasificador ART
classifier = PyTorchClassifier(
    model=wrapped,
    loss=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(model.parameters()),
    input_shape=(1, 28, 28),
    nb_classes=10,
    clip_values=(0.0, 1.0)
)

print("✅ Clasificador ART listo")

## 8. ATAQUE FGSM

In [ ]:
def evaluate_fgsm(classifier, X_test, y, epsilon=0.3):
    # EJERCICIO 5: Evalúa robustez con ataque FGSM.
    # 1. Crear ataque indicando solo estimator y eps
    # 2. Generar adversarios X_adv (usar X_test como input)
    # 3. Predecir ambos (limpios y adversarios) y calcular accuracies
    # 4. Retornar acc_clean, acc_adv
    # === INICIO SOLUCIÓN ===

    # === FIN SOLUCIÓN ===
    pass

print("Ejecutando ataque FGSM...")
clean_acc, adv_acc = evaluate_fgsm(classifier, X_test, y_test, epsilon=0.3)
print(f"\nAccuracy limpia: {clean_acc:.2f}%")
print(f"Accuracy adversaria (FGSM): {adv_acc:.2f}%")
print(f"Caída: {clean_acc - adv_acc:.2f}%")

## 9. ATAQUE PGD

In [ ]:
def evaluate_pgd(classifier, X, y, epsilon=0.3):
    # EJERCICIO 6: Evalúa robustez con ataque PGD.
    # 1. Crear ataque indicando solo estimator, eps, eps step (epsilon/10) y 40 iteraciones
    # 2. Generar y evaluar como en FGSM
    # 3. Retornar acc_clean, acc_adv
    # === INICIO SOLUCIÓN ===

    # === FIN SOLUCIÓN ===
    pass

print("Ejecutando ataque PGD...")
clean_acc_pgd, adv_acc_pgd = evaluate_pgd(classifier, X_test, y_test, epsilon=0.3)
print(f"\nAccuracy limpia: {clean_acc_pgd:.2f}%")
print(f"Accuracy adversaria (PGD): {adv_acc_pgd:.2f}%")
print(f"Caída: {clean_acc_pgd - adv_acc_pgd:.2f}%")

## 10. ADVERSARIAL TRAINING

In [ ]:
# type: ignore

from art.defences.trainer import AdversarialTrainer

# Preparar datos para adversarial training
print("\nPreparando datos para adversarial training...")

# Preparar datos de train para ART
train_art = datasets.MNIST('./data', train=True, transform=transforms.ToTensor())
train_subset_art = Subset(train_art, train_indices)
train_loader_art = DataLoader(train_subset_art, batch_size=128, shuffle=True)

# Extraer datos de entrenamiento en formato numpy
X_train = []
y_train = []
for img, label in train_subset_art:
    X_train.append(img.numpy())
    y_train.append(label)
X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"Datos de entrenamiento: {X_train.shape}")

# Crear nuevo modelo robusto
robust_model = SimpleNN().to(device)
wrapped_robust = ModelWrapper(robust_model)

# Crear clasificador para adversarial training
classifier_robust = PyTorchClassifier(
    model=wrapped_robust,
    loss=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(robust_model.parameters(), lr=0.001),
    input_shape=(1, 28, 28),
    nb_classes=10,
    clip_values=(0.0, 1.0)
)

attack_train = FastGradientMethod(estimator=classifier_robust, eps=0.1)

print("\n" + "="*70)
print("ADVERSARIAL TRAINING")
print("="*70)
print("Entrenando modelo robusto...\n")

# EJERCICIO 7: Entrenar modelo con ejemplos adversarios.
# 1. Crear AdversarialTrainer indicando classifier, attack_train y ratio=0.5
# 2. Entrenar con fit usando X_train, y_train, nb_epochs=3, batch_size=128
# === INICIO SOLUCIÓN ===

# === FIN SOLUCIÓN ===

print("\n✅ Adversarial training completado")

# Evaluar modelo robusto en datos limpios
print("\nEvaluando modelo robusto...")
acc_robust = evaluate_simple(robust_model, test_loader)
print(f"Accuracy en test limpio: {acc_robust:.2f}%")

## 11. COMPARACIÓN FINAL

In [ ]:
# Evaluar modelo robusto
print("\n" + "="*60)
print("COMPARACIÓN: BASELINE VS ROBUSTO")
print("="*60)

# Crear clasificador para modelo robusto
wrapped_robust_eval = ModelWrapper(robust_model)
classifier_robust_eval = PyTorchClassifier(
    model=wrapped_robust_eval,
    loss=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(robust_model.parameters()),
    input_shape=(1, 28, 28),
    nb_classes=10,
    clip_values=(0.0, 1.0)
)

# Evaluar ambos modelos
print("\nModelo BASELINE:")
base_clean, base_fgsm = evaluate_fgsm(classifier, X_test, y_test, 0.3)
_, base_pgd = evaluate_pgd(classifier, X_test, y_test, 0.3)
print(f"  Limpia: {base_clean:.2f}%")
print(f"  FGSM:   {base_fgsm:.2f}%")
print(f"  PGD:    {base_pgd:.2f}%")

print("\nModelo ROBUSTO:")
rob_clean, rob_fgsm = evaluate_fgsm(classifier_robust_eval, X_test, y_test, 0.3)
_, rob_pgd = evaluate_pgd(classifier_robust_eval, X_test, y_test, 0.3)
print(f"  Limpia: {rob_clean:.2f}%")
print(f"  FGSM:   {rob_fgsm:.2f}%")
print(f"  PGD:    {rob_pgd:.2f}%")

print("\n" + "="*60)
print("MEJORAS:")
print(f"  Limpia: {rob_clean - base_clean:+.2f}%")
print(f"  FGSM:   {rob_fgsm - base_fgsm:+.2f}%")
print(f"  PGD:    {rob_pgd - base_pgd:+.2f}%")
print("="*60)

In [ ]:
# type: ignore

# Visualización final
import matplotlib.pyplot as plt

metrics = ['Limpia', 'FGSM', 'PGD']
baseline_scores = [base_clean, base_fgsm, base_pgd]
robust_scores = [rob_clean, rob_fgsm, rob_pgd]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, baseline_scores, width, label='Baseline', color='lightcoral')
bars2 = ax.bar(x + width/2, robust_scores, width, label='Robusto', color='lightgreen')

ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Comparación: Modelo Baseline vs Robusto', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Añadir valores sobre barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 12. EJERCICIO 8

Responde a las siguientes preguntas en cada una de las propias celdas Markdown
del Notebook.

#### 1. Property-Based Testing

- ¿Pasaron ambos tests (ruido y consistencia)? Si alguno falló, ¿por qué crees que ocurrió?
    > [Respuesta]

- ¿Qué ventaja tiene usar Hypothesis vs. tests manuales?
    > [Respuesta]

#### 2. Validación Estadística

- ¿Cuál es el intervalo de confianza de la accuracy? ¿Es estrecho o amplio?
    > [Respuesta]

- ¿Por qué es importante reportar intervalos y no solo la media?
    > [Respuesta]


#### 3. Robustez Adversaria

- ¿Cuánto cae la accuracy bajo ataques FGSM?
    > [Respuesta]

- ¿PGD empeoró con FGSM training? ¿Por qué?
    > [Respuesta]

- ¿Cuánto mejora el adversarial training la robustez?
    > [Respuesta]


#### 4. Compromisos

- ¿El modelo robusto perdió accuracy limpia? ¿Cuánto?
    > [Respuesta]

- ¿Vale la pena el Compromisos para aplicaciones reales?
    > [Respuesta]

- ¿En qué casos recomendarías usar adversarial training?
    > [Respuesta]


## CONCLUSIONES

En esta práctica hemos:
1. Verificado invariantes del modelo con property-based testing
2. Validado rendimiento estadísticamente con intervalos de confianza
3. Expuesto la vulnerabilidad extrema a ataques adversarios
4. Logrado robustez mediante adversarial training

**Reflexión final**: La validación rigurosa —incluyendo robustez adversaria— es crítica antes de desplegar modelos en producción. Un modelo preciso pero vulnerable es un sistema inseguro.